### Hyperparameter tuning of Deep learning models in Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Installing and importing Keras-tuner
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 4.8MB/s 


In [ ]:
# loading fashion_mnist dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
# define the model
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 64-512
  hp_units = hp.Int('units', min_value = 64, max_value = 512)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [ ]:
# Instantiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 5,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
#Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.
%time tuner.search(img_train, label_train, epochs = 5, validation_data = (img_test, label_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

Trial 10 Complete [00h 00m 23s]
val_accuracy: 0.8791999816894531

Best val_accuracy So Far: 0.8791999816894531
Total elapsed time: 00h 02m 35s
INFO:tensorflow:Oracle triggered exit
CPU times: user 3min 18s, sys: 21.9 s, total: 3min 40s
Wall time: 2min 35s


In [ ]:
best_hps.get('learning_rate')

0.001

In [ ]:
best_hps.get('units')

181

In [ ]:
model = tuner.get_best_models()

In [ ]:
model[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 181)               142085    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1820      
Total params: 143,905
Trainable params: 143,905
Non-trainable params: 0
_________________________________________________________________
